<a href="https://colab.research.google.com/github/donusKim/Sales_Forecasting/blob/master/Sales_Forecasting(With_CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
#### Data Preprocessing ####

import numpy as np
import pandas as pd
import datetime
import gc
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import datetime
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras import optimizers
from datetime import date, timedelta


df=pd.read_csv("./gdrive/My Drive/롯데백화점/purchase_data.csv",encoding="ms949" )
temp=pd.read_csv("./gdrive/My Drive/롯데백화점/기온2.csv",encoding="ms949").set_index(["지점"])
rain=pd.read_csv("./gdrive/My Drive/롯데백화점/강수량2.csv",encoding="ms949").set_index(["지점"])
cpi=pd.read_csv("./gdrive/My Drive/롯데백화점/소비자물가지수.csv",encoding="ms949").set_index(["시도별"])
temp.drop("160",axis=1,inplace=True)
rain.drop("160",axis=1,inplace=True)

# Weekly Data 
import datetime
df["days"]=df['date'].apply(lambda x: datetime.datetime.strptime(str(x),'%Y-%m-%d'))
df["days"]=df["days"]-np.datetime64('2016-07-03')
df["week"]=df["days"]/7


## 취소매출 제거
df["absolute"]=abs(df["pur_amt"])
df=df.drop_duplicates(["cus_id","absolute"], keep='first')
new_df=df[["point","week","brand","absolute"]]


####점 이름 바꿔주기 ####
substitutions = {1.0: "본점",2.0: "잠실점",5.0: "부산본점",6.0: "관악점",8.0: "분당점",10.0: "영등포점",11.0: "일산점",13.0: "강남점",17.0: "창원점",22.0: "노원점",28.0: "건대스타점",333.0: "광복점"
                 ,341.0: "평촌점",344.0: "인천터미널점"}        
new_df=new_df.replace(substitutions)


new_df["week"]=new_df["week"].apply(lambda x: x.days)
new_df=new_df[(new_df["week"]!=160) & (new_df["week"]!=-1) ]

#영등포점 오입력 data 제거
new_df.drop(28391,axis=0,inplace=True)
new_df.drop(224708,axis=0,inplace=True)

#전처리 위해 reindex
new_df=new_df.set_index(["point","brand","week"]).groupby(level=[0,1,2])['absolute'].agg({('sum',np.sum)}).unstack(level=-1).fillna(0)
new_df.columns=new_df.columns.get_level_values(1)

new_df=new_df[sorted(new_df.columns)]

#관악점, 인천터미널점 제거
new_df.drop("관악점",axis=0,inplace=True)
new_df.drop("인천터미널점",axis=0,inplace=True)


temp=temp.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)
rain=rain.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)

cpi.loc["전국"]=cpi.loc["경기도"]
cpi.index=["경기북부","서울","부산","인천","경기남부","창원"]

#인천제거
rain.drop("인천",axis=0,inplace=True)
cpi.drop("인천",axis=0,inplace=True)
temp.drop("인천",axis=0,inplace=True)


point=pd.read_csv("./gdrive/My Drive/롯데백화점/point_table.csv",encoding="ms949").set_index(["point_name"])


rain_new=pd.merge(point, rain,left_on='지역', right_index=True,how='left')
temp_new=pd.merge(point, temp,left_on='지역', right_index=True,how='left')
temp_new.drop("지역",axis=1,inplace=True)
rain_new.drop("지역",axis=1,inplace=True)

temp_reindex=temp_new.reindex(new_df.index.get_level_values(0))
rain_reindex=rain_new.reindex(new_df.index.get_level_values(0))


new_df.columns=pd.date_range(start='6/30/2016', periods=160, freq='W')
temp_reindex.columns=new_df.columns
temp_reindex.columns=new_df.columns


# 3년간 기록있는 Brand만 활용
temp_reindex.index=new_df.index
rain_reindex.index=new_df.index


too_many_zero_index=new_df[((new_df['2016-07-03']==0) &(new_df['2016-07-10']==0) &(new_df['2016-07-17']==0)&(new_df['2016-07-24']==0)) | ((new_df['2019-06-30']==0) &(new_df['2019-07-07']==0) &(new_df['2019-07-14']==0)&(new_df['2019-07-21']==0)) ].index



temp_reindex=temp_reindex.loc[(new_df.index).difference(too_many_zero_index)]
rain_reindex=rain_reindex.loc[(new_df.index).difference(too_many_zero_index)]
df=new_df
new_df=new_df.loc[(new_df.index).difference(too_many_zero_index)]




#####  2016-07-03 ~~2019-07-21 까지 1주일씩(일~토)의 160주의 데이터( 주 단위 매출)

new_df.columns=pd.date_range(start='6/30/2016', periods=160, freq='D')

rain_reindex.columns=pd.date_range(start='6/30/2016', periods=160, freq='D')

temp_reindex.columns=pd.date_range(start='6/30/2016', periods=160, freq='D')


## '2016-06-30', '2016-07-01', '2016-07-02', '2016-07-03',
              # '2016-07-04', '2016-07-05', '2016-07-06', '2016-07-07',
              # '2016-07-08', '2016-07-09',
              # ...  
              # '2016-11-27', '2016-11-28', '2016-11-29', '2016-11-30',
              # '2016-12-01', '2016-12-02', '2016-12-03', '2016-12-04',
              # '2016-12-05', '2016-12-06'

In [22]:
# Create validation, test set and evaluation function

def create_dataset(df,temp,rain, timesteps, first_pred_start, is_train=True, aux_as_tensor=False, reshape_output=0):

    brand_group_mean = df.groupby('brand').mean()
    point_group_mean = df.groupby('point').mean()

    return create_dataset_part(df,temp,rain, brand_group_mean, point_group_mean, timesteps, first_pred_start, reshape_output, aux_as_tensor, is_train)

def train_generator(df,temp,rain, timesteps, first_pred_start,
    n_range=1, day_skip=7, is_train=True, batch_size=2000, aux_as_tensor=False, reshape_output=0, first_pred_start_2016=None):

    brand_group_mean = df.groupby('brand').mean()
    point_group_mean = df.groupby('point').mean()
    while 1:
        date_part = np.random.permutation(range(n_range))
        if first_pred_start_2016 is not None:
            range_diff = (first_pred_start - first_pred_start_2016).days / day_skip
            date_part = np.concat([date_part, np.random.permutation(range(range_diff, int(n_range/2) + range_diff))])

        for i in date_part:
            keep_idx = np.random.permutation(df.shape[0])[:batch_size]
            df_tmp = df.iloc[keep_idx,:]
            temp_tmp = temp.iloc[keep_idx,:]
            rain_tmp = rain.iloc[keep_idx,:]

            pred_start = first_pred_start - timedelta(days=int(day_skip*i))

            yield create_dataset_part(df_tmp,temp_tmp, rain_tmp, brand_group_mean, point_group_mean, timesteps, pred_start, reshape_output, aux_as_tensor, True)

            gc.collect()

def create_dataset_part(df, temp,rain, brand_group_mean, point_group_mean, timesteps, pred_start, reshape_output, aux_as_tensor, is_train, weight=False):

    brand_mean_df = brand_group_mean.reindex(df.index.get_level_values(1))
    point_mean_df = point_group_mean.reindex(df.index.get_level_values(0))
   
    X, y = create_xy_span(df, pred_start, timesteps, is_train)
    is0 = (X==0).astype('uint8')
    rain = rain[pd.date_range(pred_start-timedelta(days=timesteps), periods=timesteps)].values
    temp = temp[pd.date_range(pred_start-timedelta(days=timesteps), periods=timesteps)].values
    
    brand_mean, _ = create_xy_span(brand_mean_df, pred_start, timesteps, False)
    point_mean, _ = create_xy_span(point_mean_df, pred_start, timesteps, False)
    
    yearAgo, _ = create_xy_span(df, pred_start-timedelta(days=52), timesteps+4, False)
    quarterAgo, _ = create_xy_span(df, pred_start-timedelta(days=13), timesteps+4, False)
  
    if reshape_output>0:
        X = X.reshape(-1, timesteps, 1)
    if reshape_output>1:
        is0 = is0.reshape(-1, timesteps, 1)
        rain = rain.reshape(-1, timesteps, 1)
        temp = temp.reshape(-1, timesteps, 1)
        
        yearAgo = yearAgo.reshape(-1, timesteps+4, 1)
        quarterAgo = quarterAgo.reshape(-1, timesteps+4, 1)
        
        brand_mean = brand_mean.reshape(-1, timesteps, 1)
        point_mean = point_mean.reshape(-1, timesteps, 1)


    if weight: return ([X, is0,temp,rain, yearAgo, quarterAgo,brand_mean, point_mean], y)
    else: return ([X, is0,temp,rain, yearAgo, quarterAgo, brand_mean, point_mean], y)

def create_xy_span(df, pred_start, timesteps, is_train=True, shift_range=0):
    X = df[pd.date_range(pred_start-timedelta(days=timesteps), pred_start-timedelta(days=1))].values
    if (is_train & (date(2016,12,4)>pred_start)): y = df[pd.date_range(pred_start, periods=+4)].values
    else: y = None
    return X, y


 
def cal_score(Ytrue, Yfit):
	print([metrics.mean_squared_error(Ytrue, Yfit), 
	metrics.mean_squared_error(Ytrue[:,:5], Yfit[:,:5]),
	metrics.mean_squared_error(Ytrue[:,5:], Yfit[:,5:])])


In [23]:
timesteps = 13
train_data = train_generator(new_df,temp_reindex,rain_reindex, timesteps, date(2016, 11, 29),         
                                           n_range=52, day_skip=1, batch_size=20, aux_as_tensor=True, reshape_output=2)
Xval, Yval = create_dataset(new_df, temp_reindex,rain_reindex, timesteps, date(2016, 12,3),
                                     aux_as_tensor=True, reshape_output=2)
Xtest, _ = create_dataset(new_df, temp_reindex,rain_reindex, timesteps, date(2016, 12,7),
                                     aux_as_tensor=True, reshape_output=2)

In [24]:
latent_dim = 40

seq_in = Input(shape=(timesteps, 1))
is0_in = Input(shape=(timesteps, 1))
temp_in = Input(shape=(timesteps, 1))
rain_in = Input(shape=(timesteps, 1))
yearAgo_in = Input(shape=(timesteps+4, 1))
quarterAgo_in = Input(shape=(timesteps+4, 1))
brand_mean_in = Input(shape=(timesteps, 1))
point_mean_in = Input(shape=(timesteps, 1))
encode_slice = Lambda(lambda x: x[:, :timesteps, :])
x_in = concatenate([seq_in, encode_slice(yearAgo_in), encode_slice(quarterAgo_in), brand_mean_in], axis=2)


# Define network
c1 = Conv1D(latent_dim, 2, dilation_rate=1, padding='causal', activation='relu')(x_in)
conv_out = Conv1D(4, 1, activation='relu')(c1)
conv_out = Dropout(0.25)(conv_out)
conv_out = Flatten()(conv_out)
decode_slice = Lambda(lambda x: x[:, timesteps:, :])
yearAgo_pred = decode_slice(yearAgo_in)
quarterAgo_pred = decode_slice(quarterAgo_in)


# Raw sequence in results overfitting!!!
dnn_out = Dense(64, activation='relu')(Flatten()(seq_in))
dnn_out = Dropout(0.6)(dnn_out)

In [25]:
#### Train ####

x = concatenate([conv_out, dnn_out,
                 Flatten()(yearAgo_pred), Flatten()(quarterAgo_pred)])
x = Dense(64, activation='relu')(x)
x = Dropout(0.6)(x)
output = Dense(4, activation='relu')(x)
model = Model([seq_in, is0_in, temp_in,rain_in, yearAgo_in, quarterAgo_in, brand_mean_in, point_mean_in], output)
rms = optimizers.adam(lr=0.0001)
model.compile(optimizer=rms, loss='mean_squared_error')


history = model.fit_generator(train_data, steps_per_epoch=52, workers=3, use_multiprocessing=True, epochs=100, verbose=2,
                    validation_data=(Xval, Yval))


Epoch 1/100

/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'



 - 5s - loss: 60123012378939.0781 - val_loss: 38380805220538.1797
Epoch 2/100
 - 4s - loss: 66846225435096.6094 - val_loss: 38485333186373.8203
Epoch 3/100
 - 4s - loss: 56289257606852.9219 - val_loss: 38483815420276.3672
Epoch 4/100
 - 5s - loss: 62783410647355.0781 - val_loss: 37044538477474.9062
Epoch 5/100
 - 4s - loss: 52311490277691.0781 - val_loss: 35664788050292.3672
Epoch 6/100
 - 4s - loss: 56603945830872.6094 - val_loss: 34317730316288.0000
Epoch 7/100
 - 4s - loss: 49423060727335.3906 - val_loss: 33088551399051.6367
Epoch 8/100
 - 5s - loss: 57698354240905.8516 - val_loss: 31479488267729.4531
Epoch 9/100
 - 4s - loss: 51068043163175.3906 - val_loss: 29280758518132.3633
Epoch 10/100
 - 4s - loss: 54333871912487.3906 - val_loss: 27329871621771.6367
Epoch 11/100
 - 4s - loss: 53422847789528.6094 - val_loss: 26018798061009.4531
Epoch 12/100
 - 4s - loss: 46514897823586.4609 - val_loss: 26368645148299.6367
Epoch 13/100
 - 5s - loss: 39581613656536.6094 - val_loss: 2614871576948